### Нарушение согласования

Предыдущая попытка в задаче согласования рода - предсказывать род и сравнивать с ответом pymorphy

Недостаток - большое количество ошибочных предсказаний в словах, таких как "морской". 
Морской порт и цвет морской волны. Род различается, написание - нет.

Попробуем другой подход - предсказание окончаний.

Начнем с поиска ошибки согласования прилагательных

3 типа:
* новЫЙ
* хорошИЙ
* большОЙ

### 1. Построим бандл, сбалансированный по длине предложения и по корпусу.

In [4]:
from tg.grammar_ru.common import Loc
from tg.grammar_ru.corpus import CorpusReader, CorpusBuilder, BucketCorpusBalancer
from tg.grammar_ru.corpus.corpus_reader import read_data

from yo_fluq_ds import Queryable, Query, fluq

from typing import List, Union
from pathlib import Path

import math
import pandas as pd

In [3]:
CORPUS_NAMES = [
    "books.base.zip", 
    "pub.base.zip"
]
#TODO: add smth else?

CORPUS_LIST = [Loc.corpus_path/corpus_name for corpus_name in CORPUS_NAMES]

In [4]:
CORPUS_LIST

[PosixPath('/home/serhio/Data/1Education/grammar-spring/s2/grammar_ru/data-cache/corpus/books.base.zip'),
 PosixPath('/home/serhio/Data/1Education/grammar-spring/s2/grammar_ru/data-cache/corpus/pub.base.zip')]

In [5]:
def get_sentences_count(corpus_path):
    return sum([
        len(frame.groupby("sentence_id")) 
        for frame in read_data(corpus_path)
    ])

In [6]:
for i, corpus_name in enumerate(CORPUS_NAMES):
    print(f"{corpus_name} = {get_sentences_count(CORPUS_LIST[i])} sentences")

/tmp/ipykernel_12481/3628676302.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(corpus_path)
100%|██████████| 2533/2533 [00:29<00:00, 84.56it/s] 
/tmp/ipykernel_12481/3628676302.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(corpus_path)


books.base.zip = 903668 sentences


100%|██████████| 66883/66883 [08:15<00:00, 135.01it/s]

pub.base.zip = 1472084 sentences


In [7]:
LOG_BASE = math.e
BUCKET_PATH = Loc.corpus_path/"prepare/buckets/buckets.parquet"

In [8]:
BucketCorpusBalancer.build_buckets_frame(CORPUS_LIST, BUCKET_PATH, LOG_BASE)

100%|██████████| 69416/69416 [39:07<00:00, 29.57it/s]  


In [9]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
bucket,,
books.base.zip/0,"[2105037, 2635119, 2635122, 2930082, 2930089, ...",129
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
books.base.zip/5,"[189579, 784022, 2391783, 5783820, 5998276, 67...",65
pub.base.zip/0,"[51423, 71661, 461963, 1207024, 1404782, 25887...",18308
pub.base.zip/1,"[24, 33, 34, 40, 52, 53, 64, 11030, 31091, 311...",60391
pub.base.zip/2,"[3, 5, 6, 8, 10, 14, 18, 21, 25, 26, 28, 30, 3...",415067


In [10]:
BUCKET_NUMBERS = [1, 2, 3, 4]
BUCKET_LIMIT = 60_000 #Note: забили на то что в books.base.zip/4 всего 23К предложений.
# некторый дисбаланс все-же остался

In [11]:
BucketCorpusBalancer.filter_buckets_by_bucket_numbers(BUCKET_PATH, BUCKET_NUMBERS)

In [12]:
pd.read_parquet(BUCKET_PATH)

,sentences,bucket_size
books.base.zip/1,"[7, 8, 11, 15, 10711, 10730, 10731, 10745, 107...",90591
books.base.zip/2,"[1, 2, 6, 9, 12, 14, 16, 17, 18, 19, 20, 21, 2...",434207
books.base.zip/3,"[0, 3, 4, 5, 10, 13, 10712, 10716, 10718, 1072...",355499
books.base.zip/4,"[10715, 10717, 10724, 10744, 10777, 10793, 107...",23177
pub.base.zip/1,"[24, 33, 34, 40, 52, 53, 64, 11030, 31091, 311...",60391
pub.base.zip/2,"[3, 5, 6, 8, 10, 14, 18, 21, 25, 26, 28, 30, 3...",415067
pub.base.zip/3,"[0, 1, 2, 4, 7, 9, 11, 13, 15, 16, 17, 19, 20,...",820561
pub.base.zip/4,"[12, 31092, 132279, 132283, 132285, 132289, 13...",156624


In [5]:
BALANCED_PATH = Loc.corpus_path/"prepare/balanced/books&pub_60K_balanced.zip"

In [16]:
from tg.common import Logger
def balancing() -> None:
    balancer = BucketCorpusBalancer(
        buckets = pd.read_parquet(BUCKET_PATH), 
        log_base = LOG_BASE,
        bucket_limit = BUCKET_LIMIT,
    )

    CorpusBuilder.transfuse_corpus(
        sources = CORPUS_LIST,
        destination = BALANCED_PATH,
        selector = balancer
    )

In [20]:
balancing()

In [19]:
from collections import defaultdict
lengths = defaultdict(int)
for frame in read_data(BALANCED_PATH):
    for corpus_name in CORPUS_NAMES:
        lengths[corpus_name] += len(frame[frame.original_corpus_id == corpus_name].groupby("sentence_id"))

lengths

/tmp/ipykernel_12481/1361944034.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) read_data. (Use CorpusReader.read_frames_from_several_corpora)
  for frame in read_data(BALANCED_PATH):
100%|██████████| 147/147 [00:11<00:00, 12.57it/s]


defaultdict(int, {'books.base.zip': 203177, 'pub.base.zip': 240000})

In [6]:
from tg.grammar_ru import CorpusReader
from tg.grammar_ru.corpus import CorpusBuilder
from pathlib import Path

# corpus = Path('files/corpus.zip')
reader = CorpusReader(BALANCED_PATH)
reader.get_toc()

,filename,timestamp,part_index,token_count,character_count,ordinal,max_id
file_id,,,,,,,
45f6d5b5-3e14-4346-b848-003eed141143,,2023-03-13 20:24:10.177652,0,49971,220227,0,50269
20ddbb6c-2938-454a-8f12-d5114797b9fe,,2023-03-13 20:24:13.361754,1,49976,216755,1,100630
7acf76d8-cfad-4790-a82e-2902263709cd,,2023-03-13 20:24:17.309190,2,49995,208917,2,151076
ab966203-6129-4f22-9dac-fa01e965b601,,2023-03-13 20:24:24.177992,3,49987,209376,3,202229
3670730b-1f7f-4bf1-808b-33717f778f77,,2023-03-13 20:24:29.437513,4,49966,208659,4,253031
...,...,...,...,...,...,...,...
062f6711-5af1-4fa5-8ccf-5bf2d4a95bff,,2023-03-13 21:48:25.985407,142,49992,252939,142,7763628
612d244c-ae4d-4b4c-9076-e9723c63259e,,2023-03-13 21:49:10.772981,143,49984,253894,143,7818826
0bd18961-2d09-4e6c-ba2d-8786f6de8627,,2023-03-13 21:49:58.524944,144,49985,255516,144,7874300


In [9]:
reader.read_frames().first().word.head(10)

0    Собственно
1        потому
2             ,
3           что
4       перевод
5            от
6        РОСМЭН
7           был
8         очень
9          плох
Name: word, dtype: object

In [10]:
df = reader.read_frames().first()

In [23]:
#NOTE: we need nltk in requirements

In [24]:
import numpy as np
import pandas as pd

from tg.common import DataBundle
from tg.common.ml.batched_training import train_display_test_split
from tg.grammar_ru.features import PyMorphyFeaturizer

from tg.grammar_ru.corpus import ITransfuseSelector
from nltk.stem import SnowballStemmer

class AdjAgreementTrainIndexBuilder(ITransfuseSelector):
    def __init__(self):
        self.pmf = PyMorphyFeaturizer()
        self.snowball = SnowballStemmer(language="russian")
        # self.speech_part_labels = [
        #     'ADJF', 'ADJS', 'VERB', 'PRTF', 'PRTS']
        # self.gender_nums = {g: i for i, g in enumerate(
        #     ['masc', 'femn', 'neut'])}  # , 'nan'])}

    # def get_ending(normal_form)
    def select(self, source, df, toc_row):  # ~build_train_index
        df['label'] = -1
        db = DataBundle(src=df)
        self.pmf.featurize(db)  # запишет результат по ключу pymorphy
        morphed = db.data_frames['pymorphy']
        morphed.replace({np.nan: 'nan'}, inplace=True)
        return morphed
        a = morphed.POS.isin(self.speech_part_labels)
        b = (~df.word.str[0].str.isupper())
        g = morphed.gender
        c = (g != 'nan')
        df['is_target'] = a & b & c
        gender_scores_cols = [
            'gender_masc_score', 'gender_femn_score', 'gender_neut_score', 'gender_None_score']
        # 'gender_masc_score', 'gender_femn_score', 'gender_neut_score', 'gender_None_score'
        df[gender_scores_cols] = morphed[gender_scores_cols]
        df.loc[df.is_target, 'label'] = morphed[df.is_target].gender.replace(
            self.gender_nums)
        return [df]

    @staticmethod
    def build_index_from_src(src_df):
        df = src_df.loc[src_df.is_target][[
            'word_id', 'sentence_id', 'label']].copy()
        df = df.reset_index(drop=True)
        df.index.name = 'sample_id'
        df['split'] = train_display_test_split(df)
        return df


In [25]:
index_builder = AdjAgreementTrainIndexBuilder()
m = index_builder.select(_, df, _)


In [33]:
snowball = SnowballStemmer(language="russian")

In [34]:
def get_ending(normal_form):
    return snowball.stem(normal_form)

In [36]:
adjectives = m[m.POS =='ADJF']

In [37]:
adjectives["end"] = adjectives.normal_form.apply(get_ending)

/tmp/ipykernel_9771/4025221798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adjectives["end"] = adjectives.normal_form.apply(get_ending)


In [41]:
adjectives[['normal_form','end']][50:100]

,normal_form,end
word_id,,
500,тот,тот
502,сам,сам
509,который,котор
519,тихий,тих
529,говорящий,говоря
545,который,котор
579,сей,се
607,волшебный,волшебн
614,немагический,немагическ


In [12]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [14]:
morph.parse("хорошего")

[Parse(word='хорошего', tag=OpencorporaTag('ADJF,Qual neut,sing,gent'), normal_form='хороший', score=0.833333, methods_stack=((DictionaryAnalyzer(), 'хорошего', 3135, 15),)),
 Parse(word='хорошего', tag=OpencorporaTag('ADJF,Qual masc,sing,gent'), normal_form='хороший', score=0.111111, methods_stack=((DictionaryAnalyzer(), 'хорошего', 3135, 1),)),
 Parse(word='хорошего', tag=OpencorporaTag('ADJF,Qual anim,masc,sing,accs'), normal_form='хороший', score=0.055555, methods_stack=((DictionaryAnalyzer(), 'хорошего', 3135, 3),))]

In [42]:
! python3 -m pip install pymystem3

7225.15s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 KB 738.8 kB/s eta 0:00:0000:0100:01
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)


In [43]:
from pymystem3 import Mystem

In [44]:
m = Mystem()
m.lemmatize("Красивая мама красиво мыла раму")

Installing mystem to /home/serhio/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


['красивый', ' ', 'мама', ' ', 'красиво', ' ', 'мыть', ' ', 'рама', '\n']

In [48]:
text = 'Красивая мама красиво мыла раму'
import json
json.dumps(m.analyze(text), ensure_ascii=False)

'[{"analysis": [{"lex": "красивый", "wt": 1, "gr": "A=им,ед,полн,жен"}], "text": "Красивая"}, {"text": " "}, {"analysis": [{"lex": "мама", "wt": 1, "gr": "S,жен,од=им,ед"}], "text": "мама"}, {"text": " "}, {"analysis": [{"lex": "красиво", "wt": 0.8149252476, "gr": "ADV="}], "text": "красиво"}, {"text": " "}, {"analysis": [{"lex": "мыть", "wt": 0.441520999, "gr": "V,несов,пе=прош,ед,изъяв,жен"}], "text": "мыла"}, {"text": " "}, {"analysis": [{"lex": "рама", "wt": 0.9993591156, "gr": "S,жен,неод=вин,ед"}], "text": "раму"}, {"text": "\\n"}]'

In [59]:
type(m.analyze('мама сделала так, как сказал ваш начальник')[0])

dict

In [80]:
import re

In [87]:
for word_info in m.analyze('мама сделала деревянную шкатулку так, как сказал первый начальник'):
    if 'analysis' not in word_info:
        continue
    # print(word_info["analysis"][0]["gr"].split(',')[0])
    print(word_info['text'], re.split(',|=', word_info["analysis"][0]["gr"])[0])
    # print()#.split(',')[0])

мама S
сделала V
деревянную A
шкатулку S
так ADVPRO
как ADVPRO
сказал V
первый ANUM
начальник S


In [70]:
m.analyze('мама сделала так, как сказал ваш начальник')[0]["analysis"][0]["gr"].split(',')[0]

'S'

In [56]:
m.analyze('морской')


[{'analysis': [{'lex': 'морской',
    'wt': 1,
    'gr': 'A,полн=(пр,ед,жен|дат,ед,жен|род,ед,жен|твор,ед,жен|вин,ед,муж,неод|им,ед,муж)'}],
  'text': 'морской'},
 {'text': '\n'}]